In [1]:
from google.colab import ai
response = ai.generate_text("What is the capital of France?")
print(response)

ModuleNotFoundError: No module named 'google'

In [ ]:
# !pip uninstall -y ffmpeg
# !pip uninstall -y ffmpeg-python
# !pip install ffmpeg-python

: 

In [ ]:
!pip install faster-whisper flask pyngrok ffmpeg-python


: 

In [3]:
# !apt-get install -y ffmpeg

In [4]:
!lscpu

Architecture:                x86_64
  CPU op-mode(s):            32-bit, 64-bit
  Address sizes:             46 bits physical, 48 bits virtual
  Byte Order:                Little Endian
CPU(s):                      2
  On-line CPU(s) list:       0,1
Vendor ID:                   GenuineIntel
  Model name:                Intel(R) Xeon(R) CPU @ 2.00GHz
    CPU family:              6
    Model:                   85
    Thread(s) per core:      2
    Core(s) per socket:      1
    Socket(s):               1
    Stepping:                3
    BogoMIPS:                4000.28
    Flags:                   fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pg
                             e mca cmov pat pse36 clflush mmx fxsr sse sse2 ss h
                             t syscall nx pdpe1gb rdtscp lm constant_tsc rep_goo
                             d nopl xtopology nonstop_tsc cpuid tsc_known_freq p
                             ni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2ap
                   

In [5]:
!free -h


               total        used        free      shared  buff/cache   available
Mem:            12Gi       916Mi       2.9Gi       3.0Mi       8.9Gi        11Gi
Swap:             0B          0B          0B


In [6]:
!cat /proc/meminfo

MemTotal:       13286964 kB
MemFree:         3024868 kB
MemAvailable:   12019920 kB
Buffers:          188664 kB
Cached:          8740008 kB
SwapCached:            0 kB
Active:          5452200 kB
Inactive:        4168200 kB
Active(anon):       2728 kB
Inactive(anon):   692316 kB
Active(file):    5449472 kB
Inactive(file):  3475884 kB
Unevictable:           8 kB
Mlocked:               8 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               680 kB
Writeback:             0 kB
AnonPages:        691648 kB
Mapped:           417172 kB
Shmem:              3308 kB
KReclaimable:     395340 kB
Slab:             467380 kB
SReclaimable:     395340 kB
SUnreclaim:        72040 kB
KernelStack:        5648 kB
PageTables:         8180 kB
SecPageTables:         0 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6643480 kB
Committed_AS:    3112920 kB
VmallocTotal:   34359738367 kB
VmallocUsed:       91300 kB
VmallocChunk:    

In [7]:
!nvidia-smi

Sun Feb  8 18:36:45 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   58C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
import torch

print("GPU available:", torch.cuda.is_available())
print("GPU name:", torch.cuda.get_device_name(0))
print(torch.cuda.get_device_properties(0).total_memory / 1e9, "GB")


: 

In [9]:
!pip install -q cloud-tpu-client


In [10]:
import tensorflow as tf
print(tf.config.list_logical_devices('TPU'))


[]


In [11]:
import os
os.environ.get("COLAB_GPU")


'1'

In [12]:
!nvidia-smi || echo "No GPU"
!lscpu
!free -h


Sun Feb  8 18:36:56 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   57C    P0             28W /   70W |     102MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [13]:
import torch

def get_runtime_info():

    info = {}

    info["gpu_available"] = torch.cuda.is_available()

    if info["gpu_available"]:
        info["gpu_name"] = torch.cuda.get_device_name(0)
        info["gpu_memory_gb"] = round(
            torch.cuda.get_device_properties(0).total_memory / 1e9, 2
        )
    else:
        info["gpu_name"] = None

    return info

print(get_runtime_info())


{'gpu_available': True, 'gpu_name': 'Tesla T4', 'gpu_memory_gb': 15.83}


In [ ]:
from faster_whisper import WhisperModel

print("Loading Whisper model...")

# model = WhisperModel(
#     "large-v3",
#     device="cuda",
#     compute_type="int8_float16"#"float16"#
# )

model = WhisperModel(
    "small",
    compute_type="int8"
)


print("Model loaded!")


In [15]:
import numpy as np
import wave


def load_wav_audio(file_path):
    """
    Loads WAV file into float32 numpy array
    Assumes mono 16kHz audio
    """

    with wave.open(file_path, "rb") as wf:
        audio = wf.readframes(wf.getnframes())

    audio = np.frombuffer(audio, np.int16).astype("float32") / 32768.0
    return audio


import ffmpeg
import numpy as np

def load_audio_any(file_path):

    import subprocess
    import uuid

    out_file = f"/tmp/{uuid.uuid4()}.wav"

    subprocess.run([
        "ffmpeg",
        "-i", file_path,
        "-ac", "1",
        "-ar", "16000",
        "-y",
        out_file
    ], check=True)

    return out_file




def transcribe_audio(file_path):

    audio = load_audio_any(file_path)

    segments, info =  model.transcribe(
        audio,
        vad_filter=False,
        beam_size=5,
        best_of=5,
        word_timestamps=True,
        condition_on_previous_text=False
    )


    results = []

    for seg in segments:
        results.append({
            "start": seg.start,
            "end": seg.end,
            "text": seg.text.strip()
        })

    return results


In [16]:
from flask import Flask, request, jsonify
import tempfile
import os
import traceback

app = Flask(__name__)


@app.route("/health")
def health():
    return {"status": "running"}


@app.route("/transcribe_audio", methods=["POST"])
def transcribe():

    try:
        file = request.files["file"]

        tmp_file = "temp_audio.wav"
        file.save(tmp_file)

        segments = transcribe_audio(tmp_file)

        return jsonify({"segments": segments})

    except Exception as e:
        import traceback
        err = traceback.format_exc()
        print("🔥 SERVER TRACEBACK:\n", err)

        return jsonify({
            "error": str(e),
            "trace": err
        }), 500



In [17]:
!ngrok config add-authtoken 2iJeOgfAQ326CQmIZTHSeGlrcrf_6HP6dcbtFUfJHfsD6UNFZ

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [18]:
!nvidia-smi


Sun Feb  8 18:37:11 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   57C    P0             28W /   70W |     358MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [19]:
!fuser -k 5000/tcp

In [20]:
from pyngrok import ngrok
ngrok.kill()


In [ ]:
from pyngrok import ngrok

ngrok.kill()

public_url = ngrok.connect(5000)

print("PUBLIC URL:", public_url)

app.run(port=5000)


PUBLIC URL: NgrokTunnel: "https://afc5-35-240-224-210.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


: 